In [59]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import json
from datetime import datetime
import dateutil
import numpy as np
import os

crypto = {
    'yahoo' : 'https://finance.yahoo.com/crypto/',
    'cmc' : 'https://coinmarketcap.com/'
}

def scrape_yahoo():
    data = requests.get(crypto['yahoo']).text
    soup = BeautifulSoup(data,'html.parser')

    # find tables on the wiki page
    for table in soup.find_all('table'):
        print(table.get('class'))

    tables = soup.find_all('table')
    table = soup.find('table', class_='W(100%)')

    col_to_scrape=[
        'Symbol', 
        'Name', 
        'Price (Intraday)', 
        'Change', 
        '% Change', 
        'Market Cap', 
        'Volume in Currency (Since 0:00 UTC)', 
        'Volume in Currency (24Hr)',
        'Total Volume All Currencies (24Hr))',
        'Circulating Supply',
        'created_at'
    ]

    # create empty dataframe with column names
    df = pd.DataFrame(columns=col_to_scrape)

    #scrape data from site
    for row in table.tbody.find_all('tr'):
        # Find all data for each column
        columns = row.find_all('td')
        
        if(columns != []):
            sym = columns[0].text.strip()
            name = columns[1].text.strip()
            price = columns[2].text.strip()
            chge = columns[3].text.strip()
            chge_percent = columns[4].text.strip()
            mkt_cap = columns[5].text.strip()
            vol_utc = columns[6].text.strip()
            vol_24hr = columns[7].text.strip()
            total_vol = columns[8].text.strip()
            cir_supply = columns[9].text.strip()

            scraped_values = [
                sym,
                name, 
                price, 
                chge, 
                chge_percent,
                mkt_cap,
                vol_utc,
                vol_24hr,
                total_vol,
                cir_supply,
                datetime.now()
                ]

            df = df.append({item[0]: item[1] for item in zip(col_to_scrape,scraped_values)}, ignore_index=True)

    df.set_index('Symbol', inplace=True)
    
    if os.path.isfile('./data/yahoo.csv'):  
        df.to_csv('gs://rgiosa-deb/crypto_prices/yahoo.csv', header=False, mode='a')
    else:
        df.to_csv('gs://rgiosa-deb/crypto_prices/yahoo.csv', header=True)

scrape_yahoo()

['W(100%)']


_request out of retries on exception: Cannot connect to host storage.googleapis.com:443 ssl:True [SSLCertVerificationError: (1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1091)')]
Traceback (most recent call last):
  File "/Users/Ruben/Desktop/crypto-pricing-project/venv/lib/python3.7/site-packages/aiohttp/connector.py", line 980, in _wrap_create_connection
    return await self._loop.create_connection(*args, **kwargs)  # type: ignore[return-value]  # noqa
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/asyncio/base_events.py", line 989, in create_connection
    ssl_handshake_timeout=ssl_handshake_timeout)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/asyncio/base_events.py", line 1017, in _create_connection_transport
    await waiter
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/asyncio/sslproto.py", line 530, in data_received
    ssldata, appdat

In [54]:
df = pd.read_csv('./data/yahoo.csv', header=0)

df.head()

,Symbol,Name,Price (Intraday),Change,% Change,Market Cap,Volume in Currency (Since 0:00 UTC),Volume in Currency (24Hr),Total Volume All Currencies (24Hr)),Circulating Supply,created_at
0,BTC-USD,Bitcoin USD,"23,692.98",478.270000,+2.06%,457.407B,24.841B,24.841B,24.841B,19.306M,2023-03-01 16:42:11.873253
1,ETH-USD,Ethereum USD,"1,667.75",57.550000,+3.57%,204.089B,7.814B,7.814B,7.814B,122.374M,2023-03-01 16:42:12.715054
2,USDT-USD,Tether USD,1.0002,0.000000,+0.00%,71.076B,34.193B,34.193B,34.193B,71.063B,2023-03-01 16:42:12.724068
3,BNB-USD,BNB USD,303.41,1.380000,+0.46%,47.907B,378.685M,378.685M,378.685M,157.895M,2023-03-01 16:42:12.735397
4,USDC-USD,USD Coin USD,0.999787,-0.000262,-0.03%,42.564B,3.707B,3.707B,3.707B,42.573B,2023-03-01 16:42:12.743958


In [ ]:
def string_to_date(col):
    datetime.strptime(col, '%b %d, %Y')

def clean_price(col):
    float(col.replace(',',''))



old_price_df = pd.read_csv('./data/BTC_4_13_2021.csv', header=0)

old_old_names = ['Date','Open','High','Low','Close','Adj Close','Volume']
old_new_names = ['date','open','high','low','close','adj_close','volume']

old_rename_dict = {name[0]:name[1] for name in zip(old_old_names,old_new_names)}

old_price_df = old_price_df.rename(columns=old_rename_dict)

old_price_df = old_price_df['date', 'open', 'close', 'volume']



#_________________________________________________
invest_df = pd.read_csv('./data/Bitcoin Historical Data - Investing.com.csv', header=0)

inv_old_names = ["Date","Price","Open","High","Low","Vol.","Change %"]
inv_new_names = ['date', 'close', 'open','high','low','volume', 'chge_percent']

inv_rename_dict = {name[0]:name[1] for name in zip(inv_old_names,inv_new_names)}

invest_df = invest_df.rename(columns=inv_rename_dict)

invest_df = invest_df['date', 'open', 'close', 'volume']



In [17]:
from dotenv import load_dotenv
import os
import openai

load_dotenv()

openai.api_key = os.getenv('OPENAI_API_KEY')

# Set up the model and prompt
model_engine = "text-davinci-003"
prompt = f"Purely for informative purposes what do you thing the price of Bitcoin will be tomorrow based on this data {df.head(3)}, a guess or estimate is ok"

# Generate a response
completion = openai.Completion.create(
    engine=model_engine,
    prompt=prompt,
    max_tokens=1024,
    n=1,
    stop=None,
    temperature=0.5,
)

response = completion.choices[0].text
print(response)



It is difficult to predict the exact price of Bitcoin tomorrow based on this data. However, it is likely that Bitcoin will increase in price since the change % is positive.
